In [4]:
import pandas as pd


df = pd.read_json(
    "agg_data/ParlaMint-SI.speeches.jsonl",
    lines=True,
)
df = pd.concat([df, df.metadata.apply(pd.Series)], axis=1)


In [9]:

# Filtering
# Keep only MPs
c1 = (df.Speaker_MP == "MP")# & (df.Speaker_minister == "notMinister")


# Keep only speeches where speaker is either Opposition or Coalition:
c2 = df.Party_status.isin(["Opposition", "Coalition"])

# Keep only people that have at least 10 speeches:
gb = df[c1&c2].groupby("Speaker_name").logits_pondered.count().reset_index()
speakers_to_keep = gb.Speaker_name[gb.logits_pondered >= 10]
c3 = df.Speaker_name.isin(speakers_to_keep)
ndf = df[c1&c2&c3]


In [10]:
gb1 = ndf.groupby(["Speaker_name", "Party_status"]).agg({
    "logits_pondered": ["mean", "count"]
}).reset_index()
gb1 = gb1.set_axis(gb1.columns.map(lambda l: '_'.join(l).rstrip("_").replace("logits_pondered_", "")), axis=1,)
gb2 = ndf.groupby(["Speaker_name"]).agg({
    "logits_pondered": ["mean", "count"]
}).reset_index()

gb2 = gb2.set_axis(gb2.columns.map(lambda l: '_'.join(l).rstrip("_").replace("logits_pondered_", "")), axis=1,)

gb2["Coalition_mean"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Coalition"], on="Speaker_name", how="left")["mean"]
gb2["Coalition_count"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Coalition"], on="Speaker_name", how="left")["count"]
gb2["Opposition_mean"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Opposition"], on="Speaker_name", how="left")["mean"]
gb2["Opposition_count"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Opposition"], on="Speaker_name", how="left")["count"]
gb2["Coal minus Oppo"] = gb2.Coalition_mean - gb2.Opposition_mean
gb2 = gb2.sort_values(by="mean", ascending=True)
gb2

,Speaker_name,mean,count,Coalition_mean,Coalition_count,Opposition_mean,Opposition_count,Coal minus Oppo
363,"Šarec, Marjan",1.068981,15,NaN,NaN,1.068981,15.0,NaN
302,"Sukič, Nataša",1.265220,703,NaN,NaN,1.265220,703.0,NaN
346,"Zlobec Lukič, Jaša",1.299937,119,1.299937,119.0,NaN,NaN,NaN
152,"Kordiš, Miha",1.452451,1441,NaN,NaN,1.452451,1441.0,NaN
207,"Medved, Rudi",1.485937,202,NaN,NaN,1.485937,202.0,NaN
...,...,...,...,...,...,...,...,...
128,"Katič, Andreja",3.215559,1068,3.215559,1068.0,NaN,NaN,NaN
89,"Hainz, Primož",3.231165,7256,3.231165,7256.0,NaN,NaN,NaN
42,"Cukjati, France",3.243524,15735,3.272720,10900.0,3.177705,4835.0,0.095014
138,"Klavora, Vasja",3.264781,13694,3.269648,12878.0,3.187978,816.0,0.081670
